In [1]:
import os
import openai
import sys
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
import numpy as np

In [3]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']


In [4]:
#Loading the PDF - vuln
#loader = PyPDFLoader("/Users/stevenslater/Desktop/FinalProject/VulnGuidelines.pdf")

#loading my CV
loader = PyPDFLoader("/Users/stevenslater/Desktop/CV.pdf")


In [5]:
pages = loader.load()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)


In [6]:
len(pages)


1

In [7]:
page = pages[0]

In [8]:
print(page.page_content[500:1000])

ity classification, using Chain-of Thought Prompting and a newly proposed Prompt Optimisation framework. University of British Colombia – Exchange Year       Vancouver    2022-2023 • Relevant Courses: Artificial Intelligence, Functional Programming, Applied Machine Learning. • Received 2nd Highest Mark in the year for Artificial Intelligence. Hutchesons Grammar School            Glasgow        2007-2019 • Advanced Higher: Computing Science (A), Business Management (A), Mathematics (A)  • Higher:


**Chunking Text**


In [9]:
#Using recusrive splitter (played about with chunks will come back to this)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,
    separators=["\n\n", "\n", "\. ", " ", ""]
)

In [10]:
splits = r_splitter.split_documents(pages)

In [17]:
print(len(splits))

13


**Embeddings**

In [12]:
embedding = OpenAIEmbeddings()

test1 = splits[1].page_content
test2 = splits[2].page_content
print(test1)
print(test2)
embedding1 = embedding.embed_query(test1)
embedding2 = embedding.embed_query(test2)

#Checking similairities between two splits fom the PDF, will use this in retrieval later
np.dot(embedding1, embedding2)

Graduation 2024 – Projected 1st • Relevant courses: Algorithms and Data Structures, Data Science, Relational Databases, Object-Oriented Programming. • Final Year project: Leveraging Large Language Models for vulnerability classification, using Chain-of Thought Prompting and a newly proposed
a newly proposed Prompt Optimisation framework. University of British Colombia – Exchange Year       Vancouver    2022-2023 • Relevant Courses: Artificial Intelligence, Functional Programming, Applied Machine Learning. • Received 2nd Highest Mark in the year for Artificial Intelligence. Hutchesons


0.8247618940780516

**Vector Store**

In [13]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'

In [14]:
!rm -rf ./docs/chroma 

In [15]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [16]:
print(vectordb._collection.count())

13


In [18]:
#Similarity search on the vector DB
question = "What did I do at Siemens?"


In [22]:
docs = vectordb.similarity_search(question,k=1)

In [23]:
len(docs)

1

In [24]:
docs[0].page_content

"– Technology Summer Intern     Glasgow        Summer 2023 • Developed and deployed a full-stack web application using Python’s Flask framework and JavaScript, leveraging cloud computing to stream real-time diagnostic data from Siemens' Internet of Things (IoT) rail network to all employees on any"

In [35]:
#keep it saved
vectordb.persist()